#### 【 교차검증 (Cross Validation) 】

- 적은 데이터셋으로 안정적이고 신뢰성있는 모델 평가를 위한 방법
- 학습 데이터셋을 K개 분할 후 매번 다른 데이터로 검증 진행
- 교차검증 후 모델의 일반화 성능으로 여김

[1] 모듈 로딩 및 데이터 준비<hr>

In [1]:
## [1-1] 모듈 로딩
##-> 기본 모듈
import numpy as np
import pandas as pd

##-> ML 관련 모듈
from sklearn.model_selection import KFold, StratifiedGroupKFold   ## 교차검증용
from sklearn.neighbors import KNeighborsClassifier                ## 학습 알고리즘

In [2]:
## [1-2] 데이터 준비
DATA_FILE = '../Data/iris.csv'

irisDF = pd.read_csv(DATA_FILE)
irisDF.head(3)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa


[2] 데이터 전처리 및 학습 준비 <hr>

In [ ]:
## [2-1] 품종컬럼 자료형 변환
pd.options.mode.copy_on_write
irisDF.variety.astype('category')